  # **AKADEMI EDUCATION**
# **Première cohorte (2025): Science des données et intelligence artificielle**
#### *Phase 5: PROJET DE SCIENCE DES DONNÉES*

**Noms des étudiants du groupe: Riché FLEURINORD et Micka LOUIS**   
**Rythme d’apprentissage: Autonome**  
**Date de soutenance: 27 octobre 2025**  
**Noms des instructeurs: Wedter JEROME et Geovany Batista Polo LAGUERRE**  
**Lien de l’article de blog (lien du dépôt GitHub): https://github.com/richefleuriord/Bank_Customer_Churn_Prediction.git**

# *4-Modélisation*

## Modélisation et Sélection du Meilleur Modèle

La **modélisation** est le cœur du projet de *Data Science*.  
Elle a pour but d’entraîner plusieurs algorithmes de classification sur le jeu de données préparé afin de déterminer celui qui prédit le mieux le **churn des clients bancaires**.

Après la phase de préparation, les données ont été divisées en trois ensembles :
- **Train (70%)** : apprentissage des modèles.  
- **Validation (15%)** : sélection des hyperparamètres optimaux.  
- **Test (15%)** : évaluation finale et mesure de la performance réelle.

---

### Objectifs de la modélisation

- Entraîner plusieurs algorithmes performants : **Régression Logistique**, **Forêt Aléatoire**, **XGBoost**, et **Réseau de Neurones (MLP)**.  
- Mettre en place des **pipelines complets** avec encodage, normalisation et équilibrage adaptés à chaque modèle.  
- Utiliser la **validation croisée (cross-validation)** pour éviter le surapprentissage.  
- Ajuster les **hyperparamètres** à l’aide de `GridSearchCV`.  
- Comparer les modèles sur la base du **score ROC-AUC**, plus robuste que l’accuracy.  
- Sauvegarder le **meilleur modèle** pour un déploiement fluide dans l’application Streamlit.

---

### Étapes principales du processus

1. **Chargement des données préparées** : Importation des fichiers CSV (`X_train`, `X_val`, `X_test`, etc.) produits à l’étape précédente.  
2. **Définition des pipelines** : Chaque modèle est encapsulé dans un pipeline intégrant son propre prétraitement (scaling, SMOTE, pondération de classes).  
3. **Optimisation par GridSearchCV** : Recherche systématique des meilleurs hyperparamètres via validation croisée.  
4. **Évaluation sur le jeu de validation** : Mesure de la performance (accuracy, ROC-AUC).  
5. **Comparaison des résultats** pour déterminer le modèle le plus performant.  
6. **Sauvegarde du modèle final** pour le déploiement.

---

### Spécificités par modèle

| Modèle | Particularités du pipeline | Méthode d’équilibrage |
|:--------|:----------------------------|:------------------------|
| **Régression Logistique** | Normalisation avec `StandardScaler` | SMOTE |
| **Random Forest** | Modèle robuste aux variables non normalisées | `class_weight='balanced'` |
| **XGBoost** | Prise en compte du déséquilibre via `scale_pos_weight` | Pondération interne |
| **Neural Network (MLP)** | Normalisation des données requise | Aucune, équilibrage implicite |

---

In [2]:
# ============================================================
# MODELISATION OPTIMISÉE (LogReg, RF, XGBoost, NN) AVEC GRIDSEARCH
# ============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# CHARGEMENT DES DONNÉES PRÉPARÉES
# ============================================================
X_train = pd.read_csv("../Data/X_train_prepared.csv")
X_val = pd.read_csv("../Data/X_val_prepared.csv").reindex(columns=X_train.columns, fill_value=0)
X_test = pd.read_csv("../Data/X_test_prepared.csv").reindex(columns=X_train.columns, fill_value=0)

y_train = pd.read_csv("../Data/y_train_prepared.csv").values.ravel()
y_val = pd.read_csv("../Data/y_val_prepared.csv").values.ravel()
y_test = pd.read_csv("../Data/y_test_prepared.csv").values.ravel()

print(" Données préparées chargées et alignées avec succès !")

# ============================================================
# SCALE POS WEIGHT POUR XGBOOST
# ============================================================
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# ============================================================
# PIPELINES SPÉCIFIQUES
# ============================================================
num_features = [
    'CreditScore', 'Tenure', 'Balance',
    'EstimatedSalary', 'BalanceToSalaryRatio',
    'Satisfaction Score', 'Point Earned'
]
num_features = [col for col in num_features if col in X_train.columns]

pipelines = {
    "LogisticRegression": ImbPipeline(steps=[
        ('scaler', StandardScaler()),
        ('smote', SMOTE(random_state=42)),
        ('model', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "RandomForest": Pipeline(steps=[
        ('model', RandomForestClassifier(random_state=42, class_weight='balanced'))
    ]),
    "XGBoost": Pipeline(steps=[
        ('model', XGBClassifier(
            use_label_encoder=False,
            eval_metric='logloss',
            scale_pos_weight=scale_pos_weight,
            random_state=42,
            n_jobs=-1
        ))
    ]),
    "NeuralNetwork": Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', MLPClassifier(max_iter=300, random_state=42))
    ])
}

# ============================================================
# GRILLES D’HYPERPARAMÈTRES
# ============================================================
param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1, 10],
        "model__penalty": ["l2"],
        "model__solver": ["lbfgs", "liblinear"]
    },
    "RandomForest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [5, 10, 20, None],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200, 400],
        "model__max_depth": [3, 5, 6],
        "model__learning_rate": [0.01, 0.05, 0.1],
        "model__subsample": [0.8, 1.0]
    },
    "NeuralNetwork": {
        "model__hidden_layer_sizes": [(50,), (100,), (100, 50)],
        "model__activation": ["relu", "tanh"],
        "model__alpha": [0.0001, 0.001],
        "model__learning_rate_init": [0.001, 0.01]
    }
}

# ============================================================
# GRIDSEARCHCV POUR CHAQUE MODÈLE
# ============================================================
best_models = {}
results = []

for name, pipeline in pipelines.items():
    print(f"\n Entraînement et optimisation du modèle : {name}")
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[name],
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_models[name] = best_model

    print(f" Meilleurs paramètres pour {name} :", grid.best_params_)
    print(f"ROC-AUC (CV) : {grid.best_score_:.4f}")

    # Évaluation sur le jeu de validation
    y_val_pred = best_model.predict(X_val)
    y_val_proba = best_model.predict_proba(X_val)[:, 1] if hasattr(best_model, "predict_proba") else None
    val_auc = roc_auc_score(y_val, y_val_proba) if y_val_proba is not None else None
    val_acc = accuracy_score(y_val, y_val_pred)

    results.append({
        "Model": name,
        "BestParams": grid.best_params_,
        "Val Accuracy": val_acc,
        "Val ROC-AUC": val_auc
    })

# ============================================================
# RÉSULTATS ET SÉLECTION DU MEILLEUR MODÈLE
# ============================================================
results_df = pd.DataFrame(results)
print("\n Résumé des performances sur le jeu de validation :")
print(results_df)

best_model_name = results_df.sort_values(by="Val ROC-AUC", ascending=False).iloc[0]["Model"]
final_model = best_models[best_model_name]

joblib.dump(final_model, r"C:\Users\HP\course\phase_5\Models\Best_Model_Deployment.pkl")
print(f"\n Meilleur modèle sélectionné : {best_model_name} sauvegardé pour le déploiement !")


 Données préparées chargées et alignées avec succès !

 Entraînement et optimisation du modèle : LogisticRegression
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


 Meilleurs paramètres pour LogisticRegression : {'model__C': 0.01, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
ROC-AUC (CV) : 0.7603

 Entraînement et optimisation du modèle : RandomForest
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.3min finished


 Meilleurs paramètres pour RandomForest : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 200}
ROC-AUC (CV) : 0.7892

 Entraînement et optimisation du modèle : XGBoost
Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  1.1min finished


 Meilleurs paramètres pour XGBoost : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 0.8}
ROC-AUC (CV) : 0.7996

 Entraînement et optimisation du modèle : NeuralNetwork
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  8.4min finished


 Meilleurs paramètres pour NeuralNetwork : {'model__activation': 'relu', 'model__alpha': 0.001, 'model__hidden_layer_sizes': (50,), 'model__learning_rate_init': 0.001}
ROC-AUC (CV) : 0.7622

 Résumé des performances sur le jeu de validation :
                Model                                         BestParams  \
0  LogisticRegression  {'model__C': 0.01, 'model__penalty': 'l2', 'mo...   
1        RandomForest  {'model__max_depth': 10, 'model__min_samples_l...   
2             XGBoost  {'model__learning_rate': 0.1, 'model__max_dept...   
3       NeuralNetwork  {'model__activation': 'relu', 'model__alpha': ...   

   Val Accuracy  Val ROC-AUC  
0      0.695333     0.771948  
1      0.763333     0.787713  
2      0.726667     0.798801  
3      0.793333     0.759084  

 Meilleur modèle sélectionné : XGBoost sauvegardé pour le déploiement !


Le modèle XGBoost a été retenu comme modèle final de prédiction du churn client.
Il combine un excellent compromis entre performance, robustesse et capacité de généralisation.
Ce modèle sera donc sauvegardé et déployé dans l’application Streamlit pour la phase de prédiction en temps réel.